## Mission to Mars

In [47]:
# Dependencies
from bs4 import BeautifulSoup
import requests
import pymongo
from pymongo import MongoClient
from splinter import Browser
import time
import pandas as pd

In [6]:
# Initialize PyMongo to work with MongoDBs
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

# Define database and collection
db = client.mars_news_db
collection = db.items

## Scraping: NASA Mars News

In [42]:
# URL of page to be scraped
url = 'https://mars.nasa.gov/news/?page=0&per_page=15&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest'

# Retrieve page with the requests module
response = requests.get(url)
# Create BeautifulSoup object; parse with 'lxml'
soup = BeautifulSoup(response.text, 'html.parser')

# print(soup)
# Examine the results, then determine element that contains sought info
# results are returned as an iterable list
results_paragraph = soup.find_all('div', class_='image_and_description_container')
results_title = soup.find_all('div', class_='content_title')

for result in results_paragraph:
    try:
        news_p = result.find('div', class_='rollover_description_inner').text
        #print(news_p)

    except Exception as e:
        print(e)

In [45]:
for result in results_title:
    try:
        news_title = result.find('a').text
        #print(news_title)

    except Exception as e:
        print(e)

## Scraping: JPL Mars Space Images - Featured Image

In [93]:
executable_path = {'executable_path': 'chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)
url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(url)

time.sleep(4)
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

time.sleep(3)

results = soup.find_all('li', class_='slide')
#print(results)
for result in results:
    try:
        x = result.find('a', class_='fancybox')
        y = x['data-fancybox-href']
        #print(y)
        browser.quit()
    except Exception as e:
        print(e)
        browser.quit()
        
featured_image_url = "https://www.jpl.nasa.gov" + y
print(featured_image_url)

'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA22238_hires.jpg


## Scraping: Mars Weather

In [46]:
# URL of page to be scraped
url = 'https://twitter.com/marswxreport?lang=en'

# Retrieve page with the requests module
response = requests.get(url)
# Create BeautifulSoup object; parse with 'lxml'
soup = BeautifulSoup(response.text, 'html.parser')

# print(soup)
# Examine the results, then determine element that contains sought info
# results are returned as an iterable list
mars_tweets = soup.find_all('div', class_='js-tweet-text-container')
#print(mars_tweets)

for tweet in mars_tweets:
    try:
        mars_weather = tweet.find('p').text
        #print(mars_weather)

    except Exception as e:
        print(e)

## Scraping: Mars Facts

In [48]:
url = 'https://space-facts.com/mars/'

Mars_Facts = pd.read_html(url)
Mars_Facts

[                      0                              1
 0  Equatorial Diameter:                       6,792 km
 1       Polar Diameter:                       6,752 km
 2                 Mass:  6.42 x 10^23 kg (10.7% Earth)
 3                Moons:            2 (Phobos & Deimos)
 4       Orbit Distance:       227,943,824 km (1.52 AU)
 5         Orbit Period:           687 days (1.9 years)
 6  Surface Temperature:                  -153 to 20 °C
 7         First Record:              2nd millennium BC
 8          Recorded By:           Egyptian astronomers]

In [58]:
Mars_Facts_df = Mars_Facts[0]
Mars_Facts_df

,Planet_Profile_Parameter,Planet_Profile_Value
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.42 x 10^23 kg (10.7% Earth)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.52 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-153 to 20 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


In [61]:
Mars_Facts_df.columns = ['Mars_Profile_Parameter', 'Mars_Profile_Parameter_Value']


,Mars_Profile_Parameter,Mars_Profile_Parameter_Value
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.42 x 10^23 kg (10.7% Earth)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.52 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-153 to 20 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


In [150]:
Mars_Facts_df

,Mars_Profile_Parameter,Mars_Profile_Parameter_Value
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.42 x 10^23 kg (10.7% Earth)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.52 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-153 to 20 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


### DataFrame as HTML

In [62]:
#Pandas also has a `to_html` method that we can use to generate HTML tables from DataFrames.
Mars_Facts_html_table = Mars_Facts_df.to_html()
Mars_Facts_html_table

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Mars_Profile_Parameter</th>\n      <th>Mars_Profile_Parameter_Value</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>0</th>\n      <td>Equatorial Diameter:</td>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <th>1</th>\n      <td>Polar Diameter:</td>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <th>2</th>\n      <td>Mass:</td>\n      <td>6.42 x 10^23 kg (10.7% Earth)</td>\n    </tr>\n    <tr>\n      <th>3</th>\n      <td>Moons:</td>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      <th>4</th>\n      <td>Orbit Distance:</td>\n      <td>227,943,824 km (1.52 AU)</td>\n    </tr>\n    <tr>\n      <th>5</th>\n      <td>Orbit Period:</td>\n      <td>687 days (1.9 years)</td>\n    </tr>\n    <tr>\n      <th>6</th>\n      <td>Surface Temperature:</td>\n      <td>-153 to 20 °C</td>\n    </tr>\n    <tr>\n      <th>7</th>\n      <td>First Reco

In [63]:
# We need to strip unwanted newlines to clean up the table.
Mars_Facts_html_table.replace('\n', '')

'<table border="1" class="dataframe">  <thead>    <tr style="text-align: right;">      <th></th>      <th>Mars_Profile_Parameter</th>      <th>Mars_Profile_Parameter_Value</th>    </tr>  </thead>  <tbody>    <tr>      <th>0</th>      <td>Equatorial Diameter:</td>      <td>6,792 km</td>    </tr>    <tr>      <th>1</th>      <td>Polar Diameter:</td>      <td>6,752 km</td>    </tr>    <tr>      <th>2</th>      <td>Mass:</td>      <td>6.42 x 10^23 kg (10.7% Earth)</td>    </tr>    <tr>      <th>3</th>      <td>Moons:</td>      <td>2 (Phobos &amp; Deimos)</td>    </tr>    <tr>      <th>4</th>      <td>Orbit Distance:</td>      <td>227,943,824 km (1.52 AU)</td>    </tr>    <tr>      <th>5</th>      <td>Orbit Period:</td>      <td>687 days (1.9 years)</td>    </tr>    <tr>      <th>6</th>      <td>Surface Temperature:</td>      <td>-153 to 20 °C</td>    </tr>    <tr>      <th>7</th>      <td>First Record:</td>      <td>2nd millennium BC</td>    </tr>    <tr>      <th>8</th>      <td>Recorded 

In [64]:
# Saving the table directly to a file.
Mars_Facts_df.to_html('Mars_facts_table.html')

In [65]:
!open Mars_facts_table.html

In [164]:
mars_info=[]
mars_facts={}
file = open('Mars_facts_table.html', encoding='latin-1')
html = file.read()
soup = BeautifulSoup(html, 'html.parser')
for z in soup.table('td'):
    #print(z.text)
    mars_info.append(z.text.strip(':'))
    mars_facts=dict([(k, v) for k,v in zip (mars_info[::2], mars_info[1::2])])

In [165]:
mission_mars_data['mars_facts'] = mars_facts
mission_mars_data['mars_facts']

{'Equatorial Diameter': '6,792 km',
 'First Record': '2nd millennium BC',
 'Mass': '6.42 x 10^23 kg (10.7% Earth)',
 'Moons': '2 (Phobos & Deimos)',
 'Orbit Distance': '227,943,824 km (1.52 AU)',
 'Orbit Period': '687 days (1.9 years)',
 'Polar Diameter': '6,752 km',
 'Recorded By': 'Egyptian astronomers',
 'Surface Temperature': '-153 to 20 Â°C'}

## Scraping: Mars Hemisperes

In [135]:
executable_path = {'executable_path': 'chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)
url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(url)

time.sleep(4)
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

time.sleep(3)

results_title = soup.find_all('div', class_='description')

hemisphere_image_urls = []

for result in results_title:
    try:
        title = result.find('h3').text
        #print(des)
        browser.click_link_by_partial_text(title)
        time.sleep(4)
        
        html = browser.html
        soup = BeautifulSoup(html, 'html.parser')

        time.sleep(3)
        results_href = soup.find_all('div', class_='downloads')
        for result in results_href:
            img_url = result.find('a')['href']
        #print(img_url)
        
        # Run only if 'description' and 'image_url' are available
        if (title and img_url):
            # Print results
            print('-------------')
            print(title)
            print(img_url)

            # Writing values into Dictionary
            image_urls = {
                'Title': title,
                'Image_URL': img_url,
            }
        hemisphere_image_urls.append(image_urls)
        browser.visit(url)
    except Exception as e:
        print(e)
        browser.quit()
print(hemisphere_image_urls)
browser.quit()

-------------
Cerberus Hemisphere Enhanced
http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg
-------------
Schiaparelli Hemisphere Enhanced
http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg
-------------
Syrtis Major Hemisphere Enhanced
http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg
-------------
Valles Marineris Hemisphere Enhanced
http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg
[{'Title': 'Cerberus Hemisphere Enhanced', 'Image_URL': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'}, {'Title': 'Schiaparelli Hemisphere Enhanced', 'Image_URL': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'}, {'Title': 'Syrtis Major Hemisphere Enhanced', 'Image_URL': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Vik